# Exoplanet status prediction using Neural Networks

Data column meanings:
https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html#proj_disp_col
* koi_disposition:
    * CANDIDATE: A planetary candidate has passed all prior tests conducted to identify false positives, although this does not a priori mean that all possible tests have been conducted. A future test may confirm this KOI as a false positive.
    
    * FALSE POSITIVE: A false positive has failed at least one of the tests described in Batalha et al. (2012).
    
    * CONFIRMED: Cleary an exoplanet
    
A planetary candidate has passed all prior tests conducted to identify false positives, although this does not a priori mean that all possible tests have been conducted. A future test may confirm this KOI as a false positive. False positives can occur when: 1) the KOI is in reality an eclipsing binary star, 2) the Kepler light curve is contaminated by a background eclipsing binary, 3) stellar variability is confused for coherent planetary transits, or 4) instrumental artifacts are confused for coherent planetary transits.

In [1]:


import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

In [2]:
exoplanet_data = pd.read_csv("exoplanet_data.csv")

# Drop the null rows
exoplanet_data = exoplanet_data.dropna()

#Create new DataFrame for candidate expoplanets
#candidate_exoplanets = exoplanet_data[exoplanet_data['koi_disposition'] == 'CANDIDATE']

# Filter out the planets labeled as candidates
#exoplanet_data = exoplanet_data[exoplanet_data['koi_disposition'] != 'CANDIDATE']


In [3]:
exoplanet_data.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

In [4]:
# Select from the second column to the last as X
X = exoplanet_data.iloc[:, 1:]
feature_names = X.columns

# Select koi_disposition as y
y= exoplanet_data['koi_disposition']

In [5]:
# Define scaler object
scaler = MinMaxScaler().fit(X)

# Scale X
X_scaled = scaler.transform(X)

#Train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify = y, random_state = 42)

## Neural Network training and testing

What a neural network does is mostly a mistery, but it is cool to apply it anyway

In [22]:
param_grid = {'hidden_layer_sizes': [[50,50], [60,60]],
              'alpha':[0.001, 0.01, 0.1],
              'activation':['tanh', 'relu']}
grid = GridSearchCV(MLPClassifier(), param_grid, verbose=3)

# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

C:\Users\luisc\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=tanh, alpha=0.001, hidden_layer_sizes=[50, 50] .......
[CV]  activation=tanh, alpha=0.001, hidden_layer_sizes=[50, 50], score=0.875, total=   2.7s
[CV] activation=tanh, alpha=0.001, hidden_layer_sizes=[50, 50] .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV]  activation=tanh, alpha=0.001, hidden_layer_sizes=[50, 50], score=0.846, total=   2.8s
[CV] activation=tanh, alpha=0.001, hidden_layer_sizes=[50, 50] .......


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.4s remaining:    0.0s


...

[CV]  activation=relu, alpha=0.01, hidden_layer_sizes=[60, 60], score=0.883, total=   3.5s
[CV] activation=relu, alpha=0.01, hidden_layer_sizes=[60, 60] ........


C:\Users\luisc\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=relu, alpha=0.01, hidden_layer_sizes=[60, 60], score=0.882, total=   4.6s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50] .........
[CV]  activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50], score=0.869, total=   3.2s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50] .........
[CV]  activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50], score=0.872, total=   2.9s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50] .........
[CV]  activation=relu, alpha=0.1, hidden_layer_sizes=[50, 50], score=0.871, total=   2.7s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[60, 60] .........
[CV]  activation=relu, alpha=0.1, hidden_layer_sizes=[60, 60], score=0.849, total=   2.7s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[60, 60] .........
[CV]  activation=relu, alpha=0.1, hidden_layer_sizes=[60, 60], score=0.867, total=   4.5s
[CV] activation=relu, alpha=0.1, hidden_layer_sizes=[60, 60] .........
[CV]  activation=relu, alpha=0.1,

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  1.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'activation': ['tanh', 'relu'],
    

In [24]:
print('The best parameters to use are: ', grid.best_params_)

The best parameters to use are:  {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': [60, 60]}


In [26]:
network = MLPClassifier(activation = 'relu', alpha = 0.001, hidden_layer_sizes = [60,60]).fit(X_train, y_train)

print('Training set score: {:.3f}'.format(network.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(network.score(X_test, y_test)))

Training set score: 0.905
Test set score: 0.892
